# Deploying an STL Monitoring Service for the PT

In this notebook, we will deploy an STL monitoring service for the PT using `rtamt`.

This service will check whether the controller produced [PT_script-2dof_hybrid_test_bench](../hybrid-test-bench/PT_script-2dof_hybrid_test_bench.ipynb) is working as intended. In particular, it will run periodically, and:
1. Queries the time series database InfluxDB to get a batch of data from running the [PT_emulator](../hybrid-test-bench/PT_emulator.ipynb).
2. Evaluates an STL property on that specification.
3. Pushes the robustness of the specification back into InfluxDB.

In [7]:
# Check pre-requisite scripts are available.

import os

# Get the current working directory. Should be hybrid-test-bench.
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'hybrid-test-bench', 'Current directory is not hybrid-test-bench'

# Get the parent directory. Should be the root of the repository
parent_dir = current_dir

# Check that the various scripts are available.


# Do we need to create the controller.py script?


path_to_check = os.path.join(parent_dir, 'pt_emulator_service.py')
assert os.path.exists(path_to_check), f'{path_to_check} not found. Run the required notebooks in order.'

path_to_check = os.path.join(parent_dir, 'start_influxdb_rabbitmq.py')
assert os.path.exists(path_to_check), f'{path_to_check} not found. Run the required notebooks in order.'

## Starting the Hybrid Test Bench

To run the services we will need to open 3 terminals and run our scripts. Command outputs for all the terminals shouldn't indicate the scripts crashing, in order for this to work.

Follow the steps:
1. Open a terminal in [hybrid-test-bench](../hybrid-test-bench/) and run: `python start_influxdb_rabbitmq.py`

2. Open a terminal in [hybrid-test-bench](../hybrid-test-bench/) and run: `python hybrid_test_bench_data_recorder_influx.py`

3. Open a terminal in [hybrid-test-bench](../hybrid-test-bench/) and run: `python pt_emulator_service.py`

For now we don't have the contoller.py script - maybe we just add a code block which sends the message for applying the force?

4. At this stage you have the PT working, and data being recorded to the time series DB. Check in the DB management page that this is the case.


**Insert screenshot of a working InfluxDB dashboard**


Now that the system is running, we will deploy the STL monitoring service below, following the same approach.

## Deploying the Monitor

The service below queries a batch of data from the past 1h in InfluxDB, and checks that the temperature drops below the max temperature always within 60 seconds of having exceeded it. This is not always the case, so we expect to see the robustness being negative sometimes.

In [ ]:
# %%writefile stl_monitoring_service.py

# Configure python path to load the hybrid test bench modules
import sys
import os
import logging
import logging.config
import time
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
import rtamt

# Get the current working directory. Should be hybrid-test-bench.
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'hybrid-test-bench', 'Current directory is not hybrid-test-bench'

# Get the parent directory. Should be the root of the repository
parent_dir = current_dir

# The root of the repo should contain the startup folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'startup')), 'startup folder not found in the repository root'

# The root of the repo should contain the installation folder. Otherwise something went wrong during the inital setup.
assert os.path.exists(os.path.join(parent_dir, 'installation')), 'installation folder not found in the repository root'

bench_startup_dir = os.path.join(parent_dir, 'startup')

assert os.path.exists(bench_startup_dir), 'hybrid-test-bench startup directory not found'

# Add the parent directory to sys.path
sys.path.append(bench_startup_dir)

from communication.server.rabbitmq import Rabbitmq
from communication.shared.protocol import ROUTING_KEY_STATE, ROUTING_KEY_FORCES
import pt_model as pt_model

class STLMonitoringService:

    def __init__(self, rabbitmq_config, influxdb_config):

        self._rabbitmq = Rabbitmq(**rabbitmq_config)
        
        url = influxdb_config['url']
        token = influxdb_config['token']
        self._org = influxdb_config['org']
        self._bucket = influxdb_config['bucket']

        self._client = InfluxDBClient(url=url, token=token, org=self._org)

        self._l = logging.getLogger("PT_STLMonitoringService")

        # Specification
        self._spec = rtamt.StlDenseTimeSpecification()
        # Declare the variables that will correspond to the above signals.

            # Add the variable declarations that we'll use and parse.
        
        """ self._spec.declare_var('T', 'float')
        self._spec.declare_var('max_T', 'float')
        self._spec.spec = 'always((T >= max_T) implies (eventually[0:60](T <= max_T)))'
        self._spec.parse() """

    def setup(self):
        self._rabbitmq.connect_to_server()

        # Initialize the Query API
        self._query_api = self._client.query_api()
        self._write_api = self._client.write_api(write_options=SYNCHRONOUS)

        # Subscribe to any message coming from the Hybrid Test Bench physical twin.
        self._rabbitmq.subscribe(routing_key=ROUTING_KEY_STATE,
                                on_message_callback=self.process_state_sample)

        self._l.info(f"PT_STLMonitoringService setup complete.")

    def query_influxdb(self):
        # Define your Flux query: Query the relevant temperature data.
        # Feel free to adjust the start time, but leave the stop time as is.
        # We set a stop time of -3s to ensure that the data is aligned from the different measurements.


        # In our case it won't be average and max temperatures but forces and displacements. Add that to the query.


        """ flux_query = f'''
            from(bucket: "{self._bucket}")
            |> range(start: -1h, stop: -3s)
            |> filter(fn: (r) => r["_measurement"] == "emulator" or r["_measurement"] == "dtcourse_controller")
            |> filter(fn: (r) => r["_field"] == "average_temperature" or r["_field"] == "max_temperature" or r["_field"] == "min_temperature")
            |> filter(fn: (r) => r["source"] == "emulator" or r["source"] == "dtcourse_controller")
            |> aggregateWindow(every: 3s, fn: last, createEmpty: true)
            |> yield(name: "last")
        '''
        # Execute the query
        result = self._query_api.query(org=self._org, query=flux_query)

        # From the query above, the data comes in 3 tables (one per measurement).
        # Importantly, the data is aligned on the time axis, so we should be able to just transfer the data to the right arrays for ramt.
        max_temperature = []
        temperature = []
    
        for table in result:
            self._l.debug("Table:")
            self._l.debug(table)
            for record in table.records:
                ts = record.get_time().timestamp()
                self._l.debug("Record at timestamp: " + str(ts))
                self._l.debug(record)
                if record.get_field() == 'max_temperature':
                    max_temperature.append([ts, record.get_value()])
                elif record.get_field() == 'average_temperature':
                    temperature.append([ts, record.get_value()])
        
        assert len(max_temperature) == len(temperature), 'Temperature and max_temperature data not aligned.'

        return temperature, max_temperature """

    def compute_robustness(self, temperature, max_temperature):
        # Evaluate ramt on the signals and get the robustness.
        print("Evaluating ramt on the signals.")


        # Again, we will be evaluating robustness for forces and displacements, not temp.

            # robustness = self._spec.evaluate(['T', temperature], ['max_T', max_temperature])

        return None
    
    def store_robustness(self, robustness):
        # Store the robustness in the InfluxDB. Duplicate records on the same timestamp will just be updated.
        records = []
        for r in robustness:
            ts = int(r[0] * 1e9)

            records.append({
                "measurement": "robustness",
                "tags": {
                    "source": "stl_monitor"
                },
                "time": ts,
                "fields": {
                    "robustness": r[1]
                }
                })

        self._write_api.write(bucket=self._bucket, record=records)

    def process_state_sample(self, ch, method, properties, body_json):
        # Log the values received.
        self._l.info(f"Received state sample: {body_json}")
        

        # ...
        robustness = 0 # Placeholder for the robustness value.


        """ # Get the temperature history from the influxdb, and process the temperature data into signals that ramt can understand.
        temperature, max_temperature = self.query_influxdb()

        self._l.debug(f"Temperature: {temperature}")
        self._l.debug(f"Max Temperature: {max_temperature}")

        # Evaluate ramt on the signals and get the robustness.
        robustness = self.compute_robustness(temperature, max_temperature) """

        self._l.debug(f"Robustness: {robustness}")

        # Store the robustness in the InfluxDB.
        self.store_robustness(robustness)

    def start_serving(self):
        self._rabbitmq.start_consuming()

if __name__ == "__main__":
    # Get utility functions to config logging and load configuration
    from pyhocon import ConfigFactory

    # Get logging configuration
    log_conf = os.path.join(os.path.dirname(os.getcwd()), 'hybrid-test-bench', 'log.conf')
    logging.config.fileConfig(log_conf)

    # Get path to the startup.conf file used in the hybrid test bench PT & DT:
    startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'hybrid-test-bench', 'software','startup.conf')
    assert os.path.exists(startup_conf), 'startup.conf file not found'

    # The startup.conf comes from the hybrid test bench repository.
    config = ConfigFactory.parse_file(startup_conf)

    service = STLMonitoringService(rabbitmq_config=config["rabbitmq"], influxdb_config=config["influxdb"])

    service.setup()
    
    # Start the STLMonitoringService
    service.start_serving()

Running the above script from a terminal, and then opening the InfluxDB page, should give you the following results:

**Insert the two screenshots here once it works - one for data (forces & displacements) and one for robustness**

As can be seen, the robustness goes negative ...

**And explain the robustness...**